In [ ]:
import astropy.coordinates as coord
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.ndimage import gaussian_filter

# gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
from gala.units import galactic

from pyia import GaiaData

In [ ]:
galcen_frame = coord.Galactocentric(galcen_distance=8.1*u.kpc)

In [ ]:
_tbl = at.Table.read('/Users/apricewhelan/data/GaiaDR2/DR2_AllRRL_outerjoin_PS1_EBV_nocontam.csv.gz', 
                     format='ascii.csv')
_tbl.remove_column('epoch_photometry_url')

g = GaiaData(_tbl)
g = g[(g.pmra.value != 0.) & 
      (g.pmdec.value != 0.) & 
      (g.D_kpc > 0)]
c = g.get_skycoord(distance=g.D_kpc*u.kpc)

In [ ]:
galcen = c.transform_to(galcen_frame)

In [ ]:
mag_c = c.transform_to(gc.MagellanicStream)
mag_c = gc.reflex_correct(mag_c)

In [ ]:
lmc_cen = coord.SkyCoord(ra=82.5, dec=-68.25, unit=u.deg)
smc_cen = coord.SkyCoord(ra=16, dec=-73*u.deg, unit=u.deg)

In [ ]:
_tbl = g.data.copy()
_tbl['L'] = mag_c.L.degree
_tbl['B'] = mag_c.B.degree
_tbl['pmL'] = mag_c.pm_L_cosB.value
_tbl['pmB'] = mag_c.pm_B.value
_tbl.write('/Users/apricewhelan/Downloads/rrl_mag.fits')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 4))

lmc_mask = (mag_c.separation(lmc_cen) > 5*u.deg) & (mag_c.separation(smc_cen) > 2.5*u.deg)
ax.plot(mag_c.L[lmc_mask], 
        mag_c.B[lmc_mask],
        marker='o', ms=2.5, ls='none', mew=0, alpha=0.2)

ax.set_xlim(180, -180)
ax.set_ylim(-50, 50)

In [ ]:
style = dict(marker='o', ms=2.5, ls='none', mew=0, alpha=0.2)

vL = (mag_c.distance * mag_c.pm_L_cosB).to_value(u.km/u.s, u.dimensionless_angles())
vB = (mag_c.distance * mag_c.pm_B).to_value(u.km/u.s, u.dimensionless_angles())

mask = (lmc_mask &
        (np.abs(mag_c.B) < 25*u.deg) & 
        (mag_c.distance > 15*u.kpc) & 
        (vL > 50.) & (np.abs(vB) < 250))

antimask = (lmc_mask &
            ( ((mag_c.B > 25*u.deg) & (mag_c.B < (45)*u.deg)) | 
              ((mag_c.B > -(45)*u.deg) & (mag_c.B < (-25)*u.deg))) &
            (mag_c.distance > 15*u.kpc) & 
            (vL > 50.) & (np.abs(vB) < 250))

fig, axes = plt.subplots(3, 1, figsize=(10, 12),
                         sharex=True)

ax = axes[0]
ax.plot(mag_c.L.degree[mask], 
        vL[mask], **style)
ax.set_ylim(-700, 700)
ax.axhline(0.5)

ax = axes[1]
ax.plot(mag_c.L.degree[mask], 
        vB[mask], **style)
ax.set_ylim(-700, 700)

ax = axes[2]
ax.plot(mag_c.L.degree[mask], mag_c.distance[mask], **style)
ax.set_ylim(10, 75)

ax.set_xlim(180, -180)

In [ ]:
pw1_c = gc.reflex_correct(coord.SkyCoord(ra=178.8*u.deg, dec=-29.4*u.deg, distance=29*u.kpc,
                                 pm_ra_cosdec=-0.56*u.mas/u.yr,
                                 pm_dec=0.47*u.mas/u.yr,
                                 radial_velocity=249.1*u.km/u.s).transform_to(gc.MagellanicStream))
(pw1_c.pm_L_cosB * 29.*u.kpc).to(u.km/u.s, u.dimensionless_angles())

In [ ]:
# style = dict(marker='o', ms=2.5, ls='none', mew=0, alpha=0.2)
# mask = (lmc_mask &
#         (np.abs(mag_c.B) < 25*u.deg) & 
#         (mag_c.distance > 15*u.kpc) & 
#         (np.abs(mag_c.pm_B) < 2*u.mas/u.yr) &
#         (mag_c.pm_L_cosB > 0.4*u.mas/u.yr))

# mask2 = (lmc_mask &
#          (mag_c.distance > 20*u.kpc) & 
#          (np.abs(mag_c.pm_B) < 2*u.mas/u.yr) &
#          (mag_c.pm_L_cosB > 0.4*u.mas/u.yr))

# fig, axes = plt.subplots(3, 1, figsize=(10, 12),
#                          sharex=True)

# ax = axes[0]
# ax.plot(mag_c.L.degree[mask], mag_c.pm_L_cosB[mask], **style)
# ax.set_ylim(-6, 6)
# ax.axhline(0.5)

# ax = axes[1]
# ax.plot(mag_c.L.degree[mask], mag_c.pm_B[mask], **style)
# ax.set_ylim(-10, 10)

# ax = axes[2]
# ax.plot(mag_c.L.degree[mask], mag_c.distance[mask], **style)
# ax.set_ylim(10, 75)

# ax.set_xlim(180, -180)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14, 4))

ax.plot(mag_c.L[mask], mag_c.B[mask],
        marker='o', ms=2.5, ls='none', mew=0, alpha=0.2)

ax.plot(mag_c.L[antimask], mag_c.B[antimask],
        marker='o', ms=2.5, ls='none', mew=0, alpha=0.2)

ax.set_xlim(180, -180)
ax.set_ylim(-50, 50)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# ax.plot(mag_c.L[mask], mag_c.B[mask],
#         marker='o', ms=2.5, ls='none', mew=0, alpha=0.2)

H,xe,ye = np.histogram2d(mag_c.L[mask], mag_c.B[mask],
                         bins=np.arange(-30, 30+1e-5, 0.2))
H = gaussian_filter(H, 1.5)
ax.pcolormesh(xe, ye, np.arcsinh(H.T / 0.1))

ax.set_xlim(30, -30)
ax.set_ylim(-30, 30)

In [ ]:
from gala.mpl_style import hesperia, laguna

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 4))

ax.plot(mag_c.L[mask], mag_c.distance.kpc[mask],
        marker='o', ls='none', alpha=0.5, ms=2.5)

ax.set_xlim(180, -60)
ax.set_ylim(15, 80)

# ax.axhline(35., zorder=-100, color='tab:orange', alpha=0.1)

ax.scatter(64, 29., color='tab:blue', zorder=-10)

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$d$ [kpc]')

ax.xaxis.set_ticks(np.arange(-60, 180+1, 15))

fig.set_facecolor('w')
fig.tight_layout()

fig.savefig('/Users/apricewhelan/Downloads/mag_L_dist_pts.png', dpi=250)

In [ ]:
# fig, ax = plt.subplots(1, 1, figsize=(12, 6))

# ax.plot(mag_c.L[mask], mag_c.distance.kpc[mask],
#         marker='o', ls='none', alpha=0.5, ms=2.5)

# ax.set_xlim(180, -60)
# ax.set_ylim(15, 80)

# ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
# ax.set_ylabel(r'$d$ [kpc]')

# fig.set_facecolor('w')
# fig.tight_layout()

# bins = np.arange(0, 70+1, 10)
# for l, r in zip(bins[:-1], bins[1:]):
#     _mask = (mag_c.L.degree > l) & (mag_c.L.degree <= (r+5)) & mask
    
#     plt.figure()
#     plt.hist(mag_c.distance.value[_mask], bins=np.arange(15, 90+1, 2.5))
#     ax.axvline(l)
#     plt.xlim(15, 90)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

H,xe,ye = np.histogram2d(mag_c.L[mask], mag_c.distance.kpc[mask],
                         bins=(np.arange(-60, 150+1e-5, 0.2),
                               np.arange(15, 100+1e-3, 0.2)))
H = gaussian_filter(H, 4)
ax.pcolormesh(xe, ye, np.arcsinh(H.T / 0.01),
              cmap=hesperia)

ax.set_xlim(xe.max(), xe.min())
ax.set_ylim(ye.min(), ye.max())

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$d$ [kpc]')

fig.set_facecolor('w')
fig.tight_layout()

# fig.savefig('/Users/apricewhelan/Downloads/mag_L_dist.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))

H,xe,ye = np.histogram2d(mag_c.L[mask], mag_c.distance.kpc[mask],
                         bins=(np.arange(-30, 45+1e-5, 0.2),
                               np.arange(25, 80+1e-3, 0.2)))
H = gaussian_filter(H, 2.5)
ax.pcolormesh(xe, ye, np.arcsinh(H.T / 0.05),
              cmap=hesperia)

ax.set_xlim(xe.max(), xe.min())
ax.set_ylim(ye.min(), ye.max())

ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$d$ [kpc]')

fig.set_facecolor('w')
fig.tight_layout()

# fig.savefig('/Users/apricewhelan/Downloads/mag_L_dist_zoom.png', dpi=250)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

H,xe,ye = np.histogram2d(mag_c.icrs.ra.degree[mask2], 
                         mag_c.icrs.dec[mask2],
                         bins=(np.arange(0, 360+1e-5, 0.3),
                               np.arange(-90, 60+1e-3, 0.3)))
H = gaussian_filter(H, 2)
ax.pcolormesh(xe, ye, np.arcsinh(H.T / 0.008),
              cmap=hesperia)

ax.set_xlim(xe.max(), xe.min())
ax.set_ylim(ye.min(), ye.max())

# ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
# ax.set_ylabel(r'$d$ [kpc]')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

H,xe,ye = np.histogram2d(mag_c.galactic.l.wrap_at(180*u.deg).degree[mask2], 
                         mag_c.galactic.b[mask2],
                         bins=(np.arange(-180, 180+1e-5, 0.3),
                               np.arange(-90, 60+1e-3, 0.3)))
H = gaussian_filter(H, 2)
ax.pcolormesh(xe, ye, np.arcsinh(H.T / 0.008),
              cmap=hesperia)

ax.set_xlim(xe.max(), xe.min())
ax.set_ylim(ye.min(), ye.max())

# ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
# ax.set_ylabel(r'$d$ [kpc]')

fig.set_facecolor('w')
fig.tight_layout()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), 
                         sharex=True, sharey=True)

ax = axes[0]
ax.plot(mag_c.L[mask], mag_c.distance.kpc[mask],
        marker='o', ls='none', alpha=0.5, ms=2.5)

ax.set_xlim(180, -60)
ax.set_ylim(15, 80)

# ax.axhline(35., zorder=-100, color='tab:orange', alpha=0.1)

ax.scatter(64, 29., color='tab:blue', zorder=-10)


ax = axes[1]
ax.plot(mag_c.L[antimask], mag_c.distance.kpc[antimask],
        marker='o', ls='none', alpha=0.5, ms=2.5)


ax.set_xlabel(r'$L_{\rm MS}$ [deg]')
ax.set_ylabel(r'$d$ [kpc]')

ax.xaxis.set_ticks(np.arange(-60, 180+1, 15))

fig.set_facecolor('w')
fig.tight_layout()

# fig.savefig('/Users/apricewhelan/Downloads/mag_L_dist_pts.png', dpi=250)